In [11]:
# imports 
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pytz import timezone
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from pandas.plotting import register_matplotlib_converters
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from hurst import compute_Hc
register_matplotlib_converters()


In [12]:
def calculate_half_life(series):
    # Add a small constant to series
    series = series + 1e-10

    delta_series = np.log(series).diff().dropna()
    lagged_series = delta_series.shift(1).dropna()
    lagged_series = sm.add_constant(lagged_series)
    delta_series_lagged = delta_series[1:]  # Make sure delta_series_lagged is the same length as lagged_series

    # Check if delta_series_lagged and lagged_series are not empty
    if delta_series_lagged.size > 0 and lagged_series.size > 0:
        model = sm.OLS(delta_series_lagged, lagged_series)
        res = model.fit()

        # Calculate half-life
        half_life = -np.log(2) / res.params['const']
        return half_life
    else:
        return "Cannot calculate half-life because the series is empty."


In [13]:

# Function to calculate Hurst exponent
def hurst_exponent(ts):
    lags = range(2, 20)
    tau = [np.std(np.subtract(ts[lag:], ts[:-lag])) for lag in lags]
    hurst = np.polyfit(np.log(lags), np.log(tau), 1)[0]
    return hurst


In [14]:
def download_4h_data(symbols, start_date, end_date):

    try:
        data = yf.download(symbols, start=start_date, end=end_date, interval='1h')['Close']
        # Resample to 4-hour intervals
        data = data.resample('4h').last()
    except: 
        SyntaxError("Exception in downloading data")
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data.dropna(inplace=True)
    # data = data.reset_index()
    # timestamp_column_name = data.columns[0]  # Assuming the timestamp is the first column
    # data = data[[timestamp_column_name, 'Close']]
    # data.columns = ['timestamp', 'price']
    # print(symbol, start_date, end_date, interval, data.shape[0])
    return data

In [18]:
def find_cointegration(cryptos, start_date, end_date,counter, res_str=""):
    data = download_4h_data(cryptos, start_date=start_date, end_date=end_date)
    
    result = coint_johansen(data.values, det_order=0, k_ar_diff=1)
    
    eigenvectors = result.evec
    # print(eigenvectors)
    for i in range(eigenvectors.shape[1]):
        # print('**')
        stationary_series = np.dot(data.values, eigenvectors[:, i])
        result = adfuller(stationary_series)
        if result[1] < 0.05:
            hurst = hurst_exponent(stationary_series)
            half_life = calculate_half_life(pd.Series(stationary_series))
            print(*cryptos )
            print(*eigenvectors[:, i])
            print(f'P-Value: {result[1]}')
            print(f'Hurst Exponent: {hurst}')
            print(f'Half-Life: {half_life}')
            print('-------------------------------------------------------------------------------------------------------')
            res_str += str(cryptos).replace('[','').replace(']','') +'\n'
            res_str +=str(eigenvectors[:, i]).replace('[','').replace(']','') + '\n'
            res_str += "----\n"
            counter +=1
    return res_str, counter

In [27]:
res_str = ""
cryptos = ['BTC-USD', 'ETH-USD', 'USDT-USD', 'SOL-USD', 'USDC-USD', 'STETH-USD', 'BNB-USD', 'XRP-USD', 'DOGE-USD']  
start_date = datetime(2022, 11, 1, tzinfo=timezone('Asia/Tehran'))
end_date = datetime(2023, 11, 1, tzinfo=timezone('Asia/Tehran'))
counter = 0
i=0    
j=i+1
cryptos_len = len(cryptos)
while( counter <15):
    crypto_group = [cryptos[i], cryptos[j]]
    res_str, counter = find_cointegration( crypto_group, start_date, end_date, counter, res_str)     
    j += 1
    if j==cryptos_len:
        i +=1
        j =i+1

    

[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
BTC-USD USDT-USD
1.646426037523512e-05 -975.2392646241341
P-Value: 4.419942616456486e-05
Hurst Exponent: 0.202684333529463
Half-Life: Cannot calculate half-life because the series is empty.
-------------------------------------------------------------------------------------------------------
[*********************100%%**********************]  2 of 2 completed
BTC-USD SOL-USD
0.0002514638713570391 -0.23745998211005442
P-Value: 0.0021971809980979528
Hurst Exponent: 0.5760562021617769
Half-Life: -1277.8017943439072
-------------------------------------------------------------------------------------------------------
[*********************100%%**********************]  2 of 2 completed
BTC-USD USDC-USD
1.124102972257997e-05 -261.6013552074727
P-Value: 6.4742120866502e-13
Hurst Exponent: 0.20188801313085367
Half-Life: Cannot calculate half-life because t

In [28]:
print(res_str)

'BTC-USD', 'USDT-USD'
 1.64642604e-05 -9.75239265e+02
----
'BTC-USD', 'SOL-USD'
 0.00025146 -0.23745998
----
'BTC-USD', 'USDC-USD'
 1.12410297e-05 -2.61601355e+02
----
'BTC-USD', 'XRP-USD'
 2.34532559e-04 -1.50526951e+01
----
'BTC-USD', 'DOGE-USD'
8.90779426e-05 8.62475789e+01
----
'ETH-USD', 'USDT-USD'
 6.05030251e-04 -9.79749463e+02
----
'ETH-USD', 'USDC-USD'
 1.30019273e-04 -2.61425629e+02
----
'ETH-USD', 'STETH-USD'
 0.11448221 -0.11140246
----
'ETH-USD', 'DOGE-USD'
7.89136617e+01 1.53942557e-03
----
'USDT-USD', 'SOL-USD'
 2.06361921e-02 -9.73833195e+02
----
'USDT-USD', 'USDC-USD'
326.49657454 939.84355958
----
'USDT-USD', 'USDC-USD'
-67.3532053  803.95008146
----
'USDT-USD', 'STETH-USD'
 5.69620698e-04 -9.75351377e+02
----
'USDT-USD', 'BNB-USD'
 5.98294543e-03 -9.96132322e+02
----
'USDT-USD', 'XRP-USD'
978.17187506   1.65003243
----



In [29]:
file_path = "output.txt"
with open(file_path, 'w') as file:
    file.write(res_str)

In [30]:
find_cointegration(['BTC-USD', 'ETH-USD', 'SOL-USD'],start_date, end_date,0,'')

[*********************100%%**********************]  3 of 3 completed
BTC-USD ETH-USD SOL-USD
0.0004857416180584321 -0.006749029071452546 -0.17347195608095864
P-Value: 0.0007332982345492613
Hurst Exponent: 0.5492012637408994
Half-Life: Cannot calculate half-life because the series is empty.
-------------------------------------------------------------------------------------------------------


("'BTC-USD', 'ETH-USD', 'SOL-USD'\n 0.00048574 -0.00674903 -0.17347196\n----\n",
 1)

In [23]:
new_data = download_4h_data(['BTC-USD', 'ETH-USD', 'SOL-USD'],start_date,end_date)

[*********************100%%**********************]  3 of 3 completed


In [98]:
new_data ['new_price'] = new_data['BTC-USD'] * 0.0004857416180584321 + new_data["ETH-USD"] * (-0.006749029071452546 ) + \
                        new_data['SOL-USD'] * (-0.17347195608095864)

In [99]:
adfuller(new_data['new_price'])[1]

0.0007332982345492455